    trainer = PPOTrainer(
        args=training_args,
        processing_class=tokenizer,
        model=policy,
        ref_model=ref_policy,
        reward_model=reward_model,
        value_model=value_model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=peft_config,
    )



    tokenizer - straightforward
    model - pegasus loaded from disk
    ref_model - same 
    reward_model - loaded from disk
    value model - 

    train_dataset
    eval_dataset
    

In [ ]:
import shutil
import json
from datasets import Dataset
import torch
from accelerate import PartialState
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PegasusConfig,
    PegasusForConditionalGeneration,
    PegasusTokenizer,
)

from trl import (
    ModelConfig,
    PPOConfig,
    PPOTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
)
from trl.trainer.utils import SIMPLE_CHAT_TEMPLATE

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

train_path = ("train_windows.json")
val_path = ("val_windows.json")
test_path = ("test_windows.json")

def load_data(filepath):
    with open(filepath, "r") as f:
        return json.load(f)

def convert_to_dict(data):
    return {key: [d[key] for d in data] for key in data[0].keys()}

train_data = load_data(train_path)
train_data = train_data[:4227]
val_data = load_data(val_path)
val_data = val_data[:5]
test_data = load_data(test_path)
test_data = test_data[:5]

train_dict = convert_to_dict(train_data)
val_dict = convert_to_dict(val_data)
test_dict = convert_to_dict(test_data)

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)
test_dataset = Dataset.from_dict(test_dict)

def prepare_dataset(dataset, tokenizer):
    """pre-tokenize the dataset before training; only collate during training"""

    # def tokenize(element):
    #     print(element["windows"].shape)
    #     outputs = tokenizer("".join(element["windows"]), padding=False)
    #     return {"input_ids": outputs["input_ids"]}

    def tokenize(element):
        # Ensure that each item in element["windows"] is a string
        windows = element["windows"]
        if any(isinstance(item, list) for item in windows):
            # Flatten nested lists if any
            windows = [subitem for item in windows for subitem in (item if isinstance(item, list) else [item])]
        outputs = tokenizer("".join(windows), padding=False)
        return {"input_ids": outputs["input_ids"]}

    return dataset.map(tokenize,batched=True, remove_columns=dataset.column_names, num_proc=training_args.dataset_num_proc)

train_dataset = prepare_dataset(train_dataset, tokenizer)
val_dataset = prepare_dataset(val_dataset, tokenizer)
test_dataset = prepare_dataset(test_dataset, tokenizer)

print("Train dataset shape is ", train_dataset.shape)
print("Train dataset shape is ", train_dataset[0]["input_ids"].shape)
print("Train dataset shape is ", train_dataset[0]["input_ids"].shape[1])

device = "cuda" if torch.cuda.is_available() else "cpu"


value_model = AutoModelForSequenceClassification.from_pretrained("reward_model_saved", num_labels=1)
reward_model = AutoModelForSequenceClassification.from_pretrained("reward_model_saved", num_labels=1)

checkpoint = torch.load("final_model.pth", map_location=device)

model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)

ref_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
ref_model.load_state_dict(checkpoint['model_state_dict'])
ref_model = ref_model.to(device)

training_args = PPOConfig(output_dir="trainer_data")

trainer = PPOTrainer(
    args=training_args,
    processing_class=tokenizer,
    model=model,
    ref_model=ref_model,
    reward_model=reward_model,
    value_model=value_model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()




Token indices sequence length is longer than the specified maximum sequence length for this model (23212785 > 512). Running this sequence through the model will result in indexing errors                                                                          | 0/4227 [00:00<?, ? examples/s]
Map:  24%|████████████████████████████████████████████████████████▌                                                                                                                                                                                      | 1000/4227 [00:50<02:42, 19.89 examples/s]